<a href="https://colab.research.google.com/github/woojoo123/Project/blob/main/Lung_lesion_detection/Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import shutil
from glob import glob
import yaml
from tqdm.notebook import tqdm
from sklearn.model_selection import GroupKFold

In [ ]:
# 데이터 경로 설정
DATA_DIR = "/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/data/vinbigdata"
TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")  # CSV 파일 경로
IMAGE_DIR = os.path.join(DATA_DIR, "train")  # 1024px로 리사이징된 이미지 폴더
LABEL_DIR = "/content/working/vinbigdata/labels"  # YOLO 라벨 저장 경로
os.makedirs(LABEL_DIR, exist_ok=True)

In [ ]:
# YOLO 학습을 위한 데이터 로드
train_df = pd.read_csv(TRAIN_CSV)

In [ ]:
# 폐와 관련 없는 클래스 제거
EXCLUDE_CLASSES = ["Aortic enlargement", "Calcification", "Cardiomegaly"]
train_df = train_df[~train_df["class_name"].isin(EXCLUDE_CLASSES)].reset_index(drop=True)

In [ ]:
# 이미지 경로 추가
train_df["image_path"] = train_df["image_id"].apply(lambda x: os.path.join(IMAGE_DIR, x + ".png"))

In [ ]:
train_df

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height,image_path
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN,2332,2580,/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/data/vinb...
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN,2954,3159,/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/data/vinb...
2,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN,2540,3072,/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/data/vinb...
3,1c32170b4af4ce1a3030eb8167753b06,Pleural thickening,11,R9,627.0,357.0,947.0,433.0,2540,3072,/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/data/vinb...
4,0c7a38f293d5f5e4846aa4ca6db4daf1,ILD,5,R17,1347.0,245.0,2188.0,2169.0,2285,2555,/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/data/vinb...
...,...,...,...,...,...,...,...,...,...,...,...
54360,db169d0be36123bd55b866d6aa73983b,Other lesion,9,R8,6.0,670.0,272.0,1736.0,2304,2880,/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/data/vinb...
54361,936fd5cff1c058d39817a08f58b72cae,No finding,14,R1,NaN,NaN,NaN,NaN,2444,3200,/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/data/vinb...
54362,ca7e72954550eeb610fe22bf0244b7fa,No finding,14,R1,NaN,NaN,NaN,NaN,1994,2430,/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/data/vinb...
54363,aa17d5312a0fb4a2939436abca7f9579,No finding,14,R8,NaN,NaN,NaN,NaN,2048,2500,/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/data/vinb...


In [ ]:
# YOLO 형식으로 라벨 변환 및 저장
for _, row in train_df.iterrows():
    image_id = row["image_id"]
    class_id = row["class_id"]

    x_center = (row["x_min"] + row["x_max"]) / 2 / row["width"]
    y_center = (row["y_min"] + row["y_max"]) / 2 / row["height"]
    w = (row["x_max"] - row["x_min"]) / row["width"]
    h = (row["y_max"] - row["y_min"]) / row["height"]

    label_txt_path = os.path.join(LABEL_DIR, f"{image_id}.txt")

    with open(label_txt_path, "a") as f:
        f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

print("✅ YOLO 라벨 생성 완료!")

✅ YOLO 라벨 생성 완료!


In [ ]:
# 🔵 GroupKFold를 이용한 데이터 분할 (Train / Validation)
gkf = GroupKFold(n_splits=5)
train_df["fold"] = -1
for fold, (_, val_idx) in enumerate(gkf.split(train_df, groups=train_df["image_id"].tolist())):
    train_df.loc[val_idx, "fold"] = fold

In [ ]:
import numpy as np
import pandas as pd
import os
import shutil
from glob import glob
import yaml
from tqdm.notebook import tqdm
from sklearn.model_selection import GroupKFold

# 🟢 데이터 경로 설정
DATA_DIR = "/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/data/vinbigdata"
TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")  # CSV 파일 경로
IMAGE_DIR = os.path.join(DATA_DIR, "train")  # 1024px로 리사이징된 이미지 폴더
LABEL_DIR = "/content/working/vinbigdata/labels"  # YOLO 라벨 저장 경로
os.makedirs(LABEL_DIR, exist_ok=True)

# 🔵 YOLO 학습을 위한 데이터 로드
train_df = pd.read_csv(TRAIN_CSV)

# 🚨 폐와 관련 없는 클래스 제거
EXCLUDE_CLASSES = ["Aortic enlargement", "Calcification", "Cardiomegaly"]
train_df = train_df[~train_df["class_name"].isin(EXCLUDE_CLASSES)].reset_index(drop=True)

# 🔵 이미지 경로 추가
train_df["image_path"] = train_df["image_id"].apply(lambda x: os.path.join(IMAGE_DIR, x + ".png"))

# 🔵 YOLO 형식으로 라벨 변환 및 저장
for _, row in train_df.iterrows():
    image_id = row["image_id"]
    class_id = row["class_id"]

    x_center = (row["x_min"] + row["x_max"]) / 2 / row["width"]
    y_center = (row["y_min"] + row["y_max"]) / 2 / row["height"]
    w = (row["x_max"] - row["x_min"]) / row["width"]
    h = (row["y_max"] - row["y_min"]) / row["height"]

    label_txt_path = os.path.join(LABEL_DIR, f"{image_id}.txt")

    with open(label_txt_path, "a") as f:
        f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

print("✅ YOLO 라벨 생성 완료!")

# 🔵 GroupKFold를 이용한 데이터 분할 (Train / Validation)
gkf = GroupKFold(n_splits=5)
train_df["fold"] = -1
for fold, (_, val_idx) in enumerate(gkf.split(train_df, groups=train_df["image_id"].tolist())):
    train_df.loc[val_idx, "fold"] = fold

# 🔵 Train / Validation 데이터셋 분리
train_files = train_df[train_df["fold"] != 4]["image_path"].unique()
val_files = train_df[train_df["fold"] == 4]["image_path"].unique()

# 🔵 YOLO 학습을 위한 폴더 생성
OUTPUT_DIR = "/content/working/vinbigdata"
os.makedirs(os.path.join(OUTPUT_DIR, "images/train"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "images/val"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "labels/train"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "labels/val"), exist_ok=True)

# 🔵 이미지 및 라벨 파일 복사
for file in tqdm(train_files):
    shutil.copy(file, os.path.join(OUTPUT_DIR, "images/train"))
    label_file = os.path.join(LABEL_DIR, os.path.basename(file).replace(".png", ".txt"))
    if os.path.exists(label_file):
        shutil.copy(label_file, os.path.join(OUTPUT_DIR, "labels/train"))

for file in tqdm(val_files):
    shutil.copy(file, os.path.join(OUTPUT_DIR, "images/val"))
    label_file = os.path.join(LABEL_DIR, os.path.basename(file).replace(".png", ".txt"))
    if os.path.exists(label_file):
        shutil.copy(label_file, os.path.join(OUTPUT_DIR, "labels/val"))

print("✅ 데이터 분할 및 저장 완료!")

# 🔵 클래스 목록 생성
class_ids, class_names = list(zip(*set(zip(train_df["class_id"], train_df["class_name"]))))
classes = list(np.array(class_names)[np.argsort(class_ids)])

# 🔵 YOLO 학습을 위한 YAML 파일 생성
data_yaml = {
    "train": os.path.join(OUTPUT_DIR, "images/train"),
    "val": os.path.join(OUTPUT_DIR, "images/val"),
    "nc": len(classes),
    "names": classes
}

with open(os.path.join(OUTPUT_DIR, "vinbigdata.yaml"), "w") as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=False)

print("✅ YAML 파일 생성 완료!")

# 🟢 PyTorch 및 YOLO 라이브러리 불러오기
import torch
from ultralytics import YOLO
from IPython.display import clear_output

clear_output()
print(f"Setup complete. Using torch {torch.__version__} on {'GPU' if torch.cuda.is_available() else 'CPU'}")

# 🔵 YOLOv8 모델 학습 실행
model = YOLO("yolov8n.pt")  # YOLO 모델 로드
model.train(
    data=os.path.join(OUTPUT_DIR, "vinbigdata.yaml"),  # 데이터 설정
    epochs=30,  # 학습 에폭 수
    batch=10,  # 배치 크기
    imgsz=1024,  # 이미지 크기 (1024x1024)
    workers=8,  # 병렬 처리할 작업 수
    device=0  # GPU 사용 (0번 GPU)
)

print("✅ YOLO 학습 완료!")

# 수정

In [ ]:
import numpy as np
import pandas as pd
import os
import shutil
from glob import glob
import yaml
from tqdm.notebook import tqdm
from sklearn.model_selection import GroupKFold

In [ ]:
# 🟢 데이터 경로 설정
DATA_DIR = "/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/data/vinbigdata"
TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")  # CSV 파일 경로
IMAGE_DIR = os.path.join(DATA_DIR, "train")  # 1024px로 리사이징된 이미지 폴더
LABEL_DIR = "/content/working/vinbigdata/labels"  # YOLO 라벨 저장 경로
os.makedirs(LABEL_DIR, exist_ok=True)

In [ ]:
# 🔹 YOLO 학습을 위한 데이터 로드
train_df = pd.read_csv(TRAIN_CSV)

# 🚨 "No finding"(정상) 데이터 제거 (비정상 데이터만 남김)
train_df = train_df[train_df["class_name"] != "No finding"].reset_index(drop=True)

# 🚨 폐와 관련 없는 클래스 제거
EXCLUDE_CLASSES = ["Aortic enlargement", "Calcification", "Cardiomegaly"]
train_df = train_df[~train_df["class_name"].isin(EXCLUDE_CLASSES)].reset_index(drop=True)

# 🔹 이미지 경로 추가
train_df["image_path"] = train_df["image_id"].apply(lambda x: os.path.join(IMAGE_DIR, x + ".png"))

train_df

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height,image_path
0,1c32170b4af4ce1a3030eb8167753b06,Pleural thickening,11,R9,627.0,357.0,947.0,433.0,2540,3072,/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/data/vinb...
1,0c7a38f293d5f5e4846aa4ca6db4daf1,ILD,5,R17,1347.0,245.0,2188.0,2169.0,2285,2555,/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/data/vinb...
2,47ed17dcb2cbeec15182ed335a8b5a9e,Nodule/Mass,8,R9,557.0,2352.0,675.0,2484.0,2568,3353,/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/data/vinb...
3,afb6230703512afc370f236e8fe98806,Pulmonary fibrosis,13,R9,1857.0,1607.0,2126.0,2036.0,2540,3072,/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/data/vinb...
4,7c1add6833d5f0102b0d3619a1682a64,Lung Opacity,7,R10,600.0,1332.0,903.0,1523.0,2944,3014,/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/data/vinb...
...,...,...,...,...,...,...,...,...,...,...,...
22542,3b887c0550e9722c9b86b7c22ad09e11,Other lesion,9,R10,254.0,1959.0,280.0,1990.0,2630,3073,/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/data/vinb...
22543,b53d1dd80e99ca6bcef9d592f65d3321,Pleural effusion,10,R9,240.0,1550.0,562.0,2001.0,2304,2880,/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/data/vinb...
22544,26d1d5a0ef2e692c6340e74859ffdc53,Pulmonary fibrosis,13,R10,1163.0,787.0,1338.0,941.0,3072,3072,/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/data/vinb...
22545,22672ab82c290c20b86863291e25ef6c,ILD,5,R9,299.0,664.0,794.0,1508.0,2048,2500,/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/data/vinb...


In [ ]:
# 🔹 YOLO 형식으로 라벨 변환 및 저장 (NaN 값 처리 추가)
for _, row in train_df.iterrows():
    image_id = row["image_id"]
    class_id = row["class_id"]

    # NaN 값 체크 → 좌표 값이 없는 경우 스킵
    if pd.notna(row["x_min"]) and pd.notna(row["x_max"]) and pd.notna(row["y_min"]) and pd.notna(row["y_max"]):
        x_center = (row["x_min"] + row["x_max"]) / 2 / row["width"]
        y_center = (row["y_min"] + row["y_max"]) / 2 / row["height"]
        w = (row["x_max"] - row["x_min"]) / row["width"]
        h = (row["y_max"] - row["y_min"]) / row["height"]

        label_txt_path = os.path.join(LABEL_DIR, f"{image_id}.txt")

        with open(label_txt_path, "a") as f:
            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

print("✅ YOLO 라벨 생성 완료!")

✅ YOLO 라벨 생성 완료!


In [ ]:
# 🔹 GroupKFold를 이용한 데이터 분할 (Train / Validation)
gkf = GroupKFold(n_splits=5)
train_df["fold"] = -1
for fold, (_, val_idx) in enumerate(gkf.split(train_df, groups=train_df["image_id"].tolist())):
    train_df.loc[val_idx, "fold"] = fold

In [ ]:
# 🔹 Train / Validation 데이터셋 분류
train_files = train_df[train_df["fold"] != 4]["image_path"].unique()
val_files = train_df[train_df["fold"] == 4]["image_path"].unique()

In [ ]:
# 🔹 YOLO 학습을 위한 폴더 생성
OUTPUT_DIR = "/content/working/vinbigdata"
os.makedirs(os.path.join(OUTPUT_DIR, "images/train"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "images/val"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "labels/train"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "labels/val"), exist_ok=True)

In [ ]:
# 🔹 이미지 및 라벨 파일 복사
for file in tqdm(train_files):
    shutil.copy(file, os.path.join(OUTPUT_DIR, "images/train"))
    label_file = os.path.join(LABEL_DIR, os.path.basename(file).replace(".png", ".txt"))
    if os.path.exists(label_file):
        shutil.copy(label_file, os.path.join(OUTPUT_DIR, "labels/train"))

for file in tqdm(val_files):
    shutil.copy(file, os.path.join(OUTPUT_DIR, "images/val"))
    label_file = os.path.join(LABEL_DIR, os.path.basename(file).replace(".png", ".txt"))
    if os.path.exists(label_file):
        shutil.copy(label_file, os.path.join(OUTPUT_DIR, "labels/val"))

print("✅ 데이터 분할 및 저장 완료!")

  0%|          | 0/2730 [00:00<?, ?it/s]

  0%|          | 0/682 [00:00<?, ?it/s]

✅ 데이터 분할 및 저장 완료!


In [ ]:
# 🔹 클래스 목록 생성 (numpy → list 변환 추가)
class_ids, class_names = list(zip(*set(zip(train_df["class_id"], train_df["class_name"]))))
classes = list(map(str, np.array(class_names)[np.argsort(class_ids)]))  # numpy → list 변환

# 🔹 YOLO 학습을 위한 YAML 파일 생성 (numpy 데이터 제거)
data_yaml = {
    "train": os.path.join(OUTPUT_DIR, "images/train"),
    "val": os.path.join(OUTPUT_DIR, "images/val"),
    "nc": len(classes),
    "names": list(classes)  # 💡 numpy → list 변환!
}

# 🔹 YAML 저장
with open(os.path.join(OUTPUT_DIR, "vinbigdata.yaml"), "w", encoding="utf-8") as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=False)

print("✅ YAML 파일 생성 완료!")

✅ YAML 파일 생성 완료!


In [ ]:
import shutil

# 원본 디렉토리
source_dir = "/content/working/vinbigdata"

# 대상 디렉토리 (Google Drive)
target_dir = "/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/두번째"

# 기존에 같은 폴더가 있다면 삭제하고 다시 복사
shutil.rmtree(target_dir, ignore_errors=True)
shutil.copytree(source_dir, target_dir)

print(f"✅ {source_dir} → {target_dir} 복사 완료!")

✅ /content/working/vinbigdata → /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/두번째 복사 완료!


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
# 🟢 PyTorch 및 YOLO 라이브러리 불러오기
import torch
from ultralytics import YOLO
from IPython.display import clear_output

clear_output()
print(f"Setup complete. Using torch {torch.__version__} on {'GPU' if torch.cuda.is_available() else 'CPU'}")

# 🔵 YOLOv8 모델 학습 실행
model = YOLO("yolov8n.pt")  # YOLO 모델 로드
model.train(
    data='/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/두번째/vinbigdata.yaml',  # 데이터 설정
    epochs=30,  # 학습 에폭 수
    batch=10,  # 배치 크기
    imgsz=1024,  # 이미지 크기 (1024x1024)
    workers=8,  # 병렬 처리할 작업 수
    device=0  # GPU 사용 (0번 GPU)
)

print("✅ YOLO 학습 완료!")

Setup complete. Using torch 2.5.1+cu124 on GPU


100%|██████████| 6.25M/6.25M [00:00<00:00, 189MB/s]

Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)


engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/두번째/vinbigdata.yaml, epochs=30, time=None, patience=100, batch=10, imgsz=1024, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None

100%|██████████| 755k/755k [00:00<00:00, 39.9MB/s]


...

      30/30      3.34G      1.558      2.542      1.788          3       1024: 100%|██████████| 62/62 [00:12<00:00,  4.83it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  5.28it/s]

                   all        144        579      0.334      0.305      0.267       0.12



30 epochs completed in 0.129 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,793 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  3.22it/s]


                   all        144        579      0.376      0.281      0.272      0.123
         Consolidation          7         10       0.49        0.2       0.21      0.102
          Lung Opacity         20         30      0.485        0.2      0.289      0.132
           Nodule/Mass         16         48      0.321      0.438      0.315      0.161
          Other lesion         26         42      0.395      0.167      0.203     0.0955
      Pleural effusion         76        138       0.44      0.217      0.294      0.128
    Pleural thickening         51        108      0.314      0.333      0.243       0.11
          Pneumothorax         66        141      0.189      0.113     0.0835     0.0271
    Pulmonary fibrosis         22         62      0.376      0.581      0.539      0.231
Speed: 0.5ms preprocess, 5.3ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/train
✅ YOLO 학습 완료!


In [ ]:
import shutil

# 원본 디렉토리
source_dir = "/content/runs/detect/train"

# 대상 디렉토리 (Google Drive)
target_dir = "/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/두번째"

# 기존에 같은 폴더가 있다면 삭제하고 다시 복사
shutil.rmtree(target_dir, ignore_errors=True)
shutil.copytree(source_dir, target_dir)

print(f"✅ {source_dir} → {target_dir} 복사 완료!")

✅ /content/runs/detect/train → /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/두번째 복사 완료!


In [ ]:
# import numpy as np
# import pandas as pd
# import os
# import shutil
# from glob import glob
# import yaml
# from tqdm.notebook import tqdm
# from sklearn.model_selection import GroupKFold

# # 🟢 데이터 경로 설정
# DATA_DIR = "/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/data/vinbigdata"
# TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")  # CSV 파일 경로
# IMAGE_DIR = os.path.join(DATA_DIR, "train")  # 1024px로 리사이징된 이미지 폴더
# LABEL_DIR = "/content/working/vinbigdata/labels"  # YOLO 라벨 저장 경로
# os.makedirs(LABEL_DIR, exist_ok=True)

# # 🔹 YOLO 학습을 위한 데이터 로드
# train_df = pd.read_csv(TRAIN_CSV)

# # 🚨 "No finding"(정상) 데이터 제거 (비정상 데이터만 남김)
# train_df = train_df[train_df["class_name"] != "No finding"].reset_index(drop=True)

# # 🚨 폐와 관련 없는 클래스 제거
# EXCLUDE_CLASSES = ["Aortic enlargement", "Calcification", "Cardiomegaly"]
# train_df = train_df[~train_df["class_name"].isin(EXCLUDE_CLASSES)].reset_index(drop=True)

# # 🔹 이미지 경로 추가
# train_df["image_path"] = train_df["image_id"].apply(lambda x: os.path.join(IMAGE_DIR, x + ".png"))

# # 🔹 YOLO 형식으로 라벨 변환 및 저장 (NaN 값 처리 추가)
# for _, row in train_df.iterrows():
#     image_id = row["image_id"]
#     class_id = row["class_id"]

#     # NaN 값 체크 → 좌표 값이 없는 경우 스킵
#     if pd.notna(row["x_min"]) and pd.notna(row["x_max"]) and pd.notna(row["y_min"]) and pd.notna(row["y_max"]):
#         x_center = (row["x_min"] + row["x_max"]) / 2 / row["width"]
#         y_center = (row["y_min"] + row["y_max"]) / 2 / row["height"]
#         w = (row["x_max"] - row["x_min"]) / row["width"]
#         h = (row["y_max"] - row["y_min"]) / row["height"]

#         label_txt_path = os.path.join(LABEL_DIR, f"{image_id}.txt")

#         with open(label_txt_path, "a") as f:
#             f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

# print("✅ YOLO 라벨 생성 완료!")

# # 🔹 GroupKFold를 이용한 데이터 분할 (Train / Validation)
# gkf = GroupKFold(n_splits=5)
# train_df["fold"] = -1
# for fold, (_, val_idx) in enumerate(gkf.split(train_df, groups=train_df["image_id"].tolist())):
#     train_df.loc[val_idx, "fold"] = fold

# # 🔹 Train / Validation 데이터셋 분류
# train_files = train_df[train_df["fold"] != 4]["image_path"].unique()
# val_files = train_df[train_df["fold"] == 4]["image_path"].unique()

# # 🔹 YOLO 학습을 위한 폴더 생성
# OUTPUT_DIR = "/content/working/vinbigdata"
# os.makedirs(os.path.join(OUTPUT_DIR, "images/train"), exist_ok=True)
# os.makedirs(os.path.join(OUTPUT_DIR, "images/val"), exist_ok=True)
# os.makedirs(os.path.join(OUTPUT_DIR, "labels/train"), exist_ok=True)
# os.makedirs(os.path.join(OUTPUT_DIR, "labels/val"), exist_ok=True)

# # 🔹 이미지 및 라벨 파일 복사
# for file in tqdm(train_files):
#     shutil.copy(file, os.path.join(OUTPUT_DIR, "images/train"))
#     label_file = os.path.join(LABEL_DIR, os.path.basename(file).replace(".png", ".txt"))
#     if os.path.exists(label_file):
#         shutil.copy(label_file, os.path.join(OUTPUT_DIR, "labels/train"))

# for file in tqdm(val_files):
#     shutil.copy(file, os.path.join(OUTPUT_DIR, "images/val"))
#     label_file = os.path.join(LABEL_DIR, os.path.basename(file).replace(".png", ".txt"))
#     if os.path.exists(label_file):
#         shutil.copy(label_file, os.path.join(OUTPUT_DIR, "labels/val"))

# print("✅ 데이터 분할 및 저장 완료!")

# # 🔹 클래스 목록 생성
# class_ids, class_names = list(zip(*set(zip(train_df["class_id"], train_df["class_name"]))))
# classes = list(np.array(class_names)[np.argsort(class_ids)])

# # 🔹 YOLO 학습을 위한 YAML 파일 생성
# data_yaml = {
#     "train": os.path.join(OUTPUT_DIR, "images/train"),
#     "val": os.path.join(OUTPUT_DIR, "images/val"),
#     "nc": len(classes),
#     "names": classes
# }

# with open(os.path.join(OUTPUT_DIR, "vinbigdata.yaml"), "w") as outfile:
#     yaml.dump(data_yaml, outfile, default_flow_style=False)

# print("✅ YAML 파일 생성 완료!")

# 재수정

## 라이브러리 불러오기

In [ ]:
import numpy as np
import pandas as pd
import os
import shutil
from glob import glob
import yaml
from tqdm.notebook import tqdm
from sklearn.model_selection import GroupKFold

# 🟢 데이터 경로 설정
DATA_DIR = "/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/data/vinbigdata"
TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")  # CSV 파일 경로
IMAGE_DIR = os.path.join(DATA_DIR, "train")  # 1024px로 리사이징된 이미지 폴더
LABEL_DIR = "/content/working/vinbigdata/labels"  # YOLO 라벨 저장 경로
os.makedirs(LABEL_DIR, exist_ok=True)

# 🔹 YOLO 학습을 위한 데이터 로드
train_df = pd.read_csv(TRAIN_CSV)

# 🚨 "No finding"(정상) 데이터 제거 (비정상 데이터만 남김)
train_df = train_df[train_df["class_name"] != "No finding"].reset_index(drop=True)

# 🚨 폐와 관련 없는 클래스 제거
EXCLUDE_CLASSES = ["Aortic enlargement", "Calcification", "Cardiomegaly"]
train_df = train_df[~train_df["class_name"].isin(EXCLUDE_CLASSES)].reset_index(drop=True)

# 🔹 이미지 경로 추가
train_df["image_path"] = train_df["image_id"].apply(lambda x: os.path.join(IMAGE_DIR, x + ".png"))

## YOLO 형식의 라벨 생성

In [ ]:
# 🔹 class_id 재매핑 (중간에 빠진 클래스 제거)
class_ids = sorted(train_df["class_id"].unique())  # class_id를 정렬된 리스트로 가져옴
class_mapping = {old_id: new_id for new_id, old_id in enumerate(class_ids)}  # 연속된 class_id 생성
train_df["class_id"] = train_df["class_id"].map(class_mapping)  # 매핑 적용

# 🔹 YOLO 형식으로 라벨 변환 및 저장 (NaN 값 처리 추가)
for _, row in train_df.iterrows():
    image_id = row["image_id"]
    class_id = row["class_id"]

    # NaN 값 체크 → 좌표 값이 없는 경우 스킵
    if pd.notna(row["x_min"]) and pd.notna(row["x_max"]) and pd.notna(row["y_min"]) and pd.notna(row["y_max"]):
        x_center = (row["x_min"] + row["x_max"]) / 2 / row["width"]
        y_center = (row["y_min"] + row["y_max"]) / 2 / row["height"]
        w = (row["x_max"] - row["x_min"]) / row["width"]
        h = (row["y_max"] - row["y_min"]) / row["height"]

        label_txt_path = os.path.join(LABEL_DIR, f"{image_id}.txt")

        with open(label_txt_path, "a") as f:
            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

print("✅ YOLO 라벨 생성 완료!")

✅ YOLO 라벨 생성 완료!


## Train/Validation 데이터 분할

In [ ]:
# 🔹 GroupKFold를 이용한 데이터 분할 (Train / Validation)
gkf = GroupKFold(n_splits=5)
train_df["fold"] = -1
for fold, (_, val_idx) in enumerate(gkf.split(train_df, groups=train_df["image_id"].tolist())):
    train_df.loc[val_idx, "fold"] = fold

# 🔹 Train / Validation 데이터셋 분류
train_files = train_df[train_df["fold"] != 4]["image_path"].unique()
val_files = train_df[train_df["fold"] == 4]["image_path"].unique()

# 🔹 YOLO 학습을 위한 폴더 생성
OUTPUT_DIR = "/content/working/vinbigdata"
os.makedirs(os.path.join(OUTPUT_DIR, "images/train"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "images/val"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "labels/train"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "labels/val"), exist_ok=True)

# 🔹 이미지 및 라벨 파일 복사
for file in tqdm(train_files):
    shutil.copy(file, os.path.join(OUTPUT_DIR, "images/train"))
    label_file = os.path.join(LABEL_DIR, os.path.basename(file).replace(".png", ".txt"))
    if os.path.exists(label_file):
        shutil.copy(label_file, os.path.join(OUTPUT_DIR, "labels/train"))

for file in tqdm(val_files):
    shutil.copy(file, os.path.join(OUTPUT_DIR, "images/val"))
    label_file = os.path.join(LABEL_DIR, os.path.basename(file).replace(".png", ".txt"))
    if os.path.exists(label_file):
        shutil.copy(label_file, os.path.join(OUTPUT_DIR, "labels/val"))

print("✅ 데이터 분할 및 저장 완료!")

  0%|          | 0/2730 [00:00<?, ?it/s]

  0%|          | 0/682 [00:00<?, ?it/s]

✅ 데이터 분할 및 저장 완료!


## YOLO 학습을 위한 YAML 파일 생성

In [ ]:
import numpy as np
import yaml

# 🔹 클래스 목록 생성
class_ids, class_names = list(zip(*set(zip(train_df["class_id"], train_df["class_name"]))))

# ✅ numpy.array → 완전한 Python 리스트 변환 (이게 핵심)
classes = [str(name) for name in np.array(class_names)[np.argsort(class_ids)]]

# 🔹 YOLO 학습을 위한 YAML 파일 생성
data_yaml = {
    "train": os.path.join(OUTPUT_DIR, "images/train"),
    "val": os.path.join(OUTPUT_DIR, "images/val"),
    "nc": len(classes),
    "names": classes  # ✅ 이제 YOLO가 읽을 수 있는 Python 리스트임!
}

# 🔹 YAML 저장 (allow_unicode=True 추가)
yaml_path = os.path.join(OUTPUT_DIR, "vinbigdata.yaml")
with open(yaml_path, "w", encoding="utf-8") as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=False, allow_unicode=True)

print(f"✅ YAML 파일 생성 완료! 저장 위치: {yaml_path}")

✅ YAML 파일 생성 완료! 저장 위치: /content/working/vinbigdata/vinbigdata.yaml


## YOLOv8 모델 학습

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [ ]:
# 🔹 PyTorch 및 YOLO 라이브러리 불러오기
import torch
from ultralytics import YOLO
from IPython.display import clear_output

clear_output()
print(f"Setup complete. Using torch {torch.__version__} on {'GPU' if torch.cuda.is_available() else 'CPU'}")

# 🔵 YOLOv8 모델 학습 실행
model = YOLO("yolov8n.pt")  # YOLO 모델 로드
model.train(
    data=yaml_path,  # 데이터 설정
    epochs=30,  # 학습 에폭 수
    batch=10,  # 배치 크기
    imgsz=1024,  # 이미지 크기 (1024x1024)
    workers=8,  # 병렬 처리할 작업 수
    device=0,  # GPU 사용 (0번 GPU)
    project="/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n",  # 저장 경로 설정
    name="experiment_1",  # 저장 폴더 이름
)

print("✅ YOLO 학습 완료!")

Setup complete. Using torch 2.5.1+cu124 on GPU
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/working/vinbigdata/vinbigdata.yaml, epochs=30, time=None, patience=100, batch=10, imgsz=1024, save=True, save_period=-1, cache=False, device=0, workers=8, project=/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n, name=experiment_12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False

100%|██████████| 755k/755k [00:00<00:00, 113MB/s]


Overriding model.yaml nc=80 with nc=11

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 319MB/s]


AMP: checks passed ✅


...

      30/30      3.58G      1.747      2.215       1.82         97       1024: 100%|██████████| 273/273 [00:57<00:00,  4.77it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:07<00:00,  4.55it/s]

                   all        682       4509      0.375      0.315      0.286      0.124



30 epochs completed in 0.557 hours.
Optimizer stripped from /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/experiment_12/weights/last.pt, 6.4MB
Optimizer stripped from /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/experiment_12/weights/best.pt, 6.4MB

Validating /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/experiment_12/weights/best.pt...
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,793 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:08<00:00,  4.28it/s]


                   all        682       4509      0.373      0.316      0.285      0.126
           Atelectasis         25         38      0.333      0.158      0.187     0.0685
         Consolidation         64        100      0.391       0.24      0.263      0.125
                   ILD         71        203      0.481      0.286      0.319      0.164
          Infiltration        125        254      0.525      0.326      0.366      0.169
          Lung Opacity        270        498      0.415      0.317        0.3       0.13
           Nodule/Mass        158        495      0.335      0.331       0.28      0.141
          Other lesion        220        451      0.268      0.153      0.115      0.047
      Pleural effusion        204        479      0.326      0.463      0.383      0.146
    Pleural thickening        399        999      0.307      0.342      0.262     0.0979
          Pneumothorax         15         52      0.373      0.481      0.349      0.161
    Pulmonary fibrosi

In [ ]:
model = YOLO("/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/세번째/experiment_1/weights/best.pt")

model.train(
    data=yaml_path,
    epochs=20,  # 20 Epoch 진행
    batch=10,
    imgsz=1024,
    workers=8,
    device=0,
    project="/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/세번째/experiment_1/weights/best_50.pt",
    name="experiment_1",
    resume=False  # ✅ `best.pt`를 기반으로 추가 학습
)

print("✅ 추가 YOLO 학습 완료!")

Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/세번째/experiment_1/weights/best.pt, data=/content/working/vinbigdata/vinbigdata.yaml, epochs=20, time=None, patience=100, batch=10, imgsz=1024, save=True, save_period=-1, cache=False, device=0, workers=8, project=/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/세번째/experiment_1/weights/best_50.pt, name=experiment_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=

train: Scanning /content/working/vinbigdata/labels/train.cache... 2730 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2730/2730 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/working/vinbigdata/labels/val.cache... 682 images, 0 backgrounds, 0 corrupt: 100%|██████████| 682/682 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/세번째/experiment_1/weights/best_50.pt/experiment_1/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.00046875), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 1024 train, 1024 val
Using 8 dataloader workers
Logging results to /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/세번째/experiment_1/weights/best_50.pt/experiment_1
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      20/20      3.71G      1.693      2.058      1.768         91       1024: 100%|██████████| 273/273 [00:57<00:00,  4.76it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:07<00:00,  4.68it/s]

                   all        682       4509      0.368      0.312      0.281      0.125



20 epochs completed in 0.371 hours.
Optimizer stripped from /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/세번째/experiment_1/weights/best_50.pt/experiment_1/weights/last.pt, 6.3MB
Optimizer stripped from /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/세번째/experiment_1/weights/best_50.pt/experiment_1/weights/best.pt, 6.3MB

Validating /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/세번째/experiment_1/weights/best_50.pt/experiment_1/weights/best.pt...
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,793 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:08<00:00,  4.24it/s]


                   all        682       4509      0.375      0.313      0.288      0.125
           Atelectasis         25         38      0.275       0.24      0.193     0.0866
         Consolidation         64        100      0.446      0.226      0.272      0.119
                   ILD         71        203      0.366       0.34      0.329      0.173
          Infiltration        125        254       0.49      0.331      0.343      0.148
          Lung Opacity        270        498      0.424      0.318      0.304      0.126
           Nodule/Mass        158        495      0.365      0.345      0.299       0.15
          Other lesion        220        451       0.21      0.131      0.102      0.043
      Pleural effusion        204        479      0.355      0.457      0.399      0.146
    Pleural thickening        399        999      0.356      0.336      0.274      0.102
          Pneumothorax         15         52      0.455      0.346       0.33      0.147
    Pulmonary fibrosi

In [ ]:
import shutil

# 원본 디렉토리
source_dir = "/content/working/vinbigdata"

# 대상 디렉토리 (Google Drive)
target_dir = "/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/세번째"

# 기존에 같은 폴더가 있다면 삭제하고 다시 복사
shutil.rmtree(target_dir, ignore_errors=True)
shutil.copytree(source_dir, target_dir)

print(f"✅ {source_dir} → {target_dir} 복사 완료!")

✅ /content/working/vinbigdata → /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/세번째 복사 완료!


## YOLO 모델 예측 실행 (테스트)

In [ ]:
# 🔹 학습한 YOLO 모델을 불러와 예측 수행
model = YOLO("/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/experiment_12/weights/best.pt")  # 학습된 모델 로드

# 🔹 예측 실행
results = model.predict(
    source=os.path.join(OUTPUT_DIR, "images/val"),  # 검증 이미지 폴더
    conf=0.25,  # confidence threshold
    iou=0.5,  # IoU threshold
    save=True  # 결과 저장
)

print("✅ YOLO 예측 완료!")


image 1/682 /content/working/vinbigdata/images/val/001d127bad87592efe45a5c7678f8b8d.png: 1024x1024 1 Pulmonary fibrosis, 13.2ms
image 2/682 /content/working/vinbigdata/images/val/0061cf6d35e253b6e7f03940592cc35e.png: 1024x1024 1 Pulmonary fibrosis, 13.4ms
image 3/682 /content/working/vinbigdata/images/val/009d837e29ba400e03856cf8d6a5b545.png: 1024x1024 (no detections), 13.5ms
image 4/682 /content/working/vinbigdata/images/val/010018c93ed33ae56ed048ee54867e46.png: 1024x1024 1 Pulmonary fibrosis, 13.2ms
image 5/682 /content/working/vinbigdata/images/val/01546d3e6175ceaabd7d92f0c566579d.png: 1024x1024 1 Nodule/Mass, 13.2ms
image 6/682 /content/working/vinbigdata/images/val/0162dad8330007f8f0daf43bcf4033f1.png: 1024x1024 (no detections), 13.2ms
image 7/682 /content/working/vinbigdata/images/val/01ee6e560f083255a630c41bba779405.png: 1024x1024 1 Nodule/Mass, 13.1ms
image 8/682 /content/working/vinbigdata/images/val/02222df826791117f315ecb2b58c5b69.png: 1024x1024 (no detections), 13.2ms
imag

## 증강

In [ ]:
import yaml

# 새로운 YAML 파일 경로
yaml_path = "/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8m/vinbigdata.yaml"

# 올바른 데이터 경로로 업데이트
yaml_data = {
    "train": "/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/세번째/images/train",
    "val": "/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/세번째/images/val",
    "nc": 11,  # 클래스 개수 (수정 필요 시 변경)
    "names": [
        "Atelectasis",
        "Consolidation",
        "ILD",
        "Infiltration",
        "Lung Opacity",
        "Nodule/Mass",
        "Other lesion",
        "Pleural effusion",
        "Pleural thickening",
        "Pneumothorax",
        "Pulmonary fibrosis"
    ]
}

# 수정된 YAML 파일 저장
with open(yaml_path, "w") as f:
    yaml.dump(yaml_data, f, default_flow_style=False, allow_unicode=True)

print(f"✅ YAML 파일이 업데이트되었습니다: {yaml_path}")

✅ YAML 파일이 업데이트되었습니다: /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8m/vinbigdata.yaml


In [ ]:
# 🔹 PyTorch 및 YOLO 라이브러리 불러오기
import torch
from ultralytics import YOLO
from IPython.display import clear_output

clear_output()
print(f"Setup complete. Using torch {torch.__version__} on {'GPU' if torch.cuda.is_available() else 'CPU'}")

# 🔵 YOLOv8 모델 학습 실행
model = YOLO("yolov8m.pt")  # YOLO 모델 로드
model.train(
    data='/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8m/vinbigdata.yaml',
    epochs=30,  # 학습 에폭 증가
    batch=16,  # 배치 크기 증가
    imgsz=1024,  # 이미지 크기 유지
    workers=8,  # 병렬 처리
    device=0,  # GPU 사용
    augment=True,  # ✅ YOLO 내장 증강 추가
    project="/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8m",
    name="experiment_full_augmented"  # 실험명 변경
)

Setup complete. Using torch 2.5.1+cu124 on GPU
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8m/vinbigdata.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=1024, save=True, save_period=-1, cache=False, device=0, workers=8, project=/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8m, name=experiment_full_augmented2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes

train: Scanning /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/세번째/labels/train... 2730 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2730/2730 [08:57<00:00,  5.08it/s]


train: New cache created: /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/세번째/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/세번째/labels/val... 682 images, 0 backgrounds, 0 corrupt: 100%|██████████| 682/682 [02:33<00:00,  4.45it/s]


val: New cache created: /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/세번째/labels/val.cache
Plotting labels to /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8m/experiment_full_augmented2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000667, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 1024 train, 1024 val
Using 8 dataloader workers
Logging results to /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8m/experiment_full_augmented2
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        682       4509      0.404      0.338      0.318      0.141



30 epochs completed in 1.959 hours.
Optimizer stripped from /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8m/experiment_full_augmented2/weights/last.pt, 52.1MB
Optimizer stripped from /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8m/experiment_full_augmented2/weights/best.pt, 52.1MB

Validating /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8m/experiment_full_augmented2/weights/best.pt...
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,846,129 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:43<00:00,  1.99s/it]


                   all        682       4509      0.396      0.331      0.321      0.146
           Atelectasis         25         38      0.266      0.211        0.2     0.0734
         Consolidation         64        100      0.512       0.29      0.343      0.157
                   ILD         71        203      0.399      0.335      0.362      0.191
          Infiltration        125        254      0.504      0.363      0.409      0.191
          Lung Opacity        270        498      0.399      0.279      0.309      0.136
           Nodule/Mass        158        495      0.418      0.392      0.345       0.17
          Other lesion        220        451      0.265      0.186      0.135     0.0586
      Pleural effusion        204        479      0.376      0.469      0.401      0.161
    Pleural thickening        399        999      0.355      0.312      0.277      0.106
          Pneumothorax         15         52       0.48      0.385      0.389      0.213
    Pulmonary fibrosi

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f7182b09390>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.

In [ ]:
model = YOLO("/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8m/experiment_full_augmented2/weights/best.pt")

model.train(
    data='/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8m/vinbigdata.yaml',
    epochs=20,  # 20 Epoch 추가 진행
    batch=10,
    imgsz=1024,
    workers=8,
    device=0,
    project="/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8m",
    name="20epoch 추가학습"
)

print("✅ 추가 YOLO 학습 완료!")

Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8m/experiment_full_augmented2/weights/best.pt, data=/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8m/vinbigdata.yaml, epochs=20, time=None, patience=100, batch=10, imgsz=1024, save=True, save_period=-1, cache=False, device=0, workers=8, project=/content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8m, name=20epoch 추가학습, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visuali

train: Scanning /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/세번째/labels/train.cache... 2730 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2730/2730 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8n/세번째/labels/val.cache... 682 images, 0 backgrounds, 0 corrupt: 100%|██████████| 682/682 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8m/20epoch 추가학습/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000667, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.00046875), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 1024 train, 1024 val
Using 8 dataloader workers
Logging results to /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8m/20epoch 추가학습
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      20/20      9.92G      1.496      1.642      1.721         91       1024: 100%|██████████| 273/273 [03:35<00:00,  1.26it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:21<00:00,  1.65it/s]

                   all        682       4509      0.379      0.352      0.311      0.139



20 epochs completed in 1.332 hours.
Optimizer stripped from /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8m/20epoch 추가학습/weights/last.pt, 52.1MB
Optimizer stripped from /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8m/20epoch 추가학습/weights/best.pt, 52.1MB

Validating /content/drive/MyDrive/프로젝트/딥러닝 프로젝트/YOLO/YOLOv8m/20epoch 추가학습/weights/best.pt...
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,846,129 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:20<00:00,  1.68it/s]


                   all        682       4509      0.377      0.354      0.312      0.139
           Atelectasis         25         38      0.321      0.263      0.252      0.115
         Consolidation         64        100      0.412       0.32      0.314       0.14
                   ILD         71        203       0.43      0.382      0.358      0.182
          Infiltration        125        254      0.518      0.323       0.37      0.171
          Lung Opacity        270        498      0.365      0.331       0.28      0.125
           Nodule/Mass        158        495      0.377      0.408       0.32      0.156
          Other lesion        220        451      0.242      0.201      0.133     0.0547
      Pleural effusion        204        479      0.373      0.465      0.405      0.147
    Pleural thickening        399        999      0.336      0.384      0.289      0.109
          Pneumothorax         15         52      0.404      0.404      0.364      0.198
    Pulmonary fibrosi